In [1]:
import numpy             as np
import scipy             as sp
import matplotlib.pyplot as plt
import matplotlib
import time
from numba import njit

from tqdm            import tqdm
from scipy           import signal

from propagation import received_power, quad_distribute_power, propagation_delay, gen_qrx_onlyclocked

from parametermeasurement import generate_clocks, measure_range_roberts, measure_bearing

from positioning import classicalfix_directbearing_mle, classicalfix_diffbearing_mle, \
                        classicalfix_directrange_mle, classicalfix_diffrange_mle, \
                        classicalfix_directbearing_crlb, classicalfix_diffbearing_crlb, \
                        classicalfix_directrange_crlb, classicalfix_diffrange_crlb

from simulation import Simulation, generate_simulation_clocks

In [2]:
iterations = 1000

### world parameters
sim = Simulation(weather        = "fog", 
                 temperature    = 298,   # [K], circuit temperature
                 daynight       = "night", 
                 rxconfig       = "optics/qrx_planoconvex.npz", 
                 txconfig       = "optics/tx_lambertian_20deg_2W.npz", 
                 istxlambertian = True, 
                 f_adc_clk      = 1.0e7,  # [Hz] ADC measurement clock freq
                 f_e            = 1.0e6,  # [Hz] emitted wave freq, left TX
                 f_sim          = 1.0e10) # [Hz] simulation master clock freq

L_ego    = 1.6 # [m]
L_target = 1.5 # [m] arbitrary selection, just to make it different than the ego. This is not known, so it's not used in the algo

### build coordinates
x = np.concatenate((np.linspace(-1,-3, 5), np.linspace(-3, -2, 5), np.linspace(-2,3, 5), np.linspace(3,0, 5)))
y = np.concatenate((np.linspace(3, 10, 5), np.linspace(10, 16, 5), np.linspace(16,14, 5), np.linspace(14,7, 5)))
hdg = np.zeros(x.shape) # parallel vehicles

numpts = hdg.shape[0]

# simulation for each heat map point takes 1 vlp cycle
vlp_rate    = 100
t_sim_stop  = 1/vlp_rate
s_adc_clock = generate_simulation_clocks(t_sim_stop, sim.f_simulation, sim.f_adc_clock, gen_clocked_only=True)

f_vehicle   = 1e3; # [Hz], trajectory simulation clock rate, fixed by trajectory generation
s_veh_clock = generate_simulation_clocks(t_sim_stop, sim.f_simulation, f_vehicle, gen_clocked_only=True)

### Define full simulation run as a function

In [3]:
def simrun(x, y, hdg, s_adc_clock, sim, iterations):
    ### simulate received power
    pwr_txL_to_rxL = received_power(x, y, np.zeros_like(y), hdg, sim)
    pwr_txL_to_rxR = received_power(x-L_ego, y, np.zeros_like(y), hdg, sim)
    pwr_txR_to_rxL = received_power(x+L_target, y, np.zeros_like(y), hdg, sim)
    pwr_txR_to_rxR = received_power(x+L_target-L_ego, y, np.zeros_like(y), hdg, sim)
    
    shared_pwr_txL_to_rxL = quad_distribute_power(x, y, 0, sim.rx_config_bundle["f_QRX"], pwr_txL_to_rxL)
    shared_pwr_txL_to_rxR = quad_distribute_power(x-L_ego, y, 0, sim.rx_config_bundle["f_QRX"], pwr_txL_to_rxR)
    shared_pwr_txR_to_rxL = quad_distribute_power(x+L_target, y, 0, sim.rx_config_bundle["f_QRX"], pwr_txR_to_rxL)
    shared_pwr_txR_to_rxR = quad_distribute_power(x+L_target-L_ego, y, 0, sim.rx_config_bundle["f_QRX"], pwr_txR_to_rxR)
    
    delay_txL_to_rxL = propagation_delay(x, y, sim.lightspeed)
    delay_txL_to_rxR = propagation_delay(x-L_ego, y, sim.lightspeed)
    delay_txR_to_rxL = propagation_delay(x+L_target, y, sim.lightspeed)
    delay_txR_to_rxR = propagation_delay(x+L_target-L_ego, y, sim.lightspeed)
    
    ### run simulation
    aoa_rxL_txL     = np.zeros((iterations)); aoa_rxR_txL     = np.zeros((iterations))
    aoa_rxL_txR     = np.zeros((iterations)); aoa_rxR_txR     = np.zeros((iterations))
    delaoa_rxLR_txL = np.zeros((iterations)); delaoa_rxLR_txR = np.zeros((iterations))
    d_rxL_txL       = np.zeros((iterations)); d_rxR_txL       = np.zeros((iterations))
    d_rxL_txR       = np.zeros((iterations)); d_rxR_txR       = np.zeros((iterations))
    deld_rxLR_txL   = np.zeros((iterations)); deld_rxLR_txR   = np.zeros((iterations))

    est_x_directbearing = np.zeros((iterations)); est_y_directbearing = np.zeros((iterations))
    est_x_diffbearing   = np.zeros((iterations)); est_y_diffbearing   = np.zeros((iterations))
    est_x_directrange   = np.zeros((iterations)); est_y_directrange   = np.zeros((iterations))
    est_x_diffrange     = np.zeros((iterations)); est_y_diffrange     = np.zeros((iterations))
    
    for j in range(0,iterations):
        smp_lo = 0
        smp_hi = int(f_vehicle/vlp_rate)
        numticks = int(f_vehicle/vlp_rate)
        step_time       = s_veh_clock[smp_lo:smp_hi] 
        simulation_time = s_adc_clock[0:int(sim.f_adc_clock/f_vehicle)]

        rxLL, rxLR, rxRL, rxRR, delays = gen_qrx_onlyclocked(shared_pwr_txL_to_rxL, shared_pwr_txL_to_rxR, 
                                                             shared_pwr_txR_to_rxL, shared_pwr_txR_to_rxR,
                                                             delay_txL_to_rxL, delay_txL_to_rxR, 
                                                             delay_txR_to_rxL, delay_txR_to_rxR, sim, step_time, 
                                                             simulation_time, smp_lo, smp_hi);
        (rxLA_txL, rxLB_txL, rxLC_txL, rxLD_txL) = rxLL
        (rxLA_txR, rxLB_txR, rxLC_txR, rxLD_txR) = rxLR
        (rxRA_txL, rxRB_txL, rxRC_txL, rxRD_txL) = rxRL
        (rxRA_txR, rxRB_txR, rxRC_txR, rxRD_txR) = rxRR 
        (delayLL_sigTime, delayLR_sigTime, delayRL_sigTime, delayRR_sigTime) = delays

        del rxLL, rxLR, rxRL, rxRR, delays 

        # initial transmitted signal is known (mark the +, that's because the signal does a roundtrip
        # which is 1 delay earlier than echo transmission from target vehicle)
        wav_rxL_txL = np.sin(2*np.pi*sim.f_emitted*(simulation_time + delayLL_sigTime))
        wav_rxR_txL = np.sin(2*np.pi*sim.f_emitted*(simulation_time + delayLR_sigTime))
        wav_rxL_txR = np.sin(2*np.pi*sim.f_emitted*(simulation_time + delayRL_sigTime))
        wav_rxR_txR = np.sin(2*np.pi*sim.f_emitted*(simulation_time + delayRR_sigTime))

        d_rxL_txL[j] = measure_range_roberts(rxLA_txL + rxLB_txL + rxLC_txL + rxLD_txL, wav_rxL_txL, sim.lightspeed, sim.f_emitted)
        d_rxR_txL[j] = measure_range_roberts(rxRA_txL + rxRB_txL + rxRC_txL + rxRD_txL, wav_rxR_txL, sim.lightspeed, sim.f_emitted)
        d_rxL_txR[j] = measure_range_roberts(rxLA_txR + rxLB_txR + rxLC_txR + rxLD_txR, wav_rxL_txR, sim.lightspeed, sim.f_emitted)
        d_rxR_txR[j] = measure_range_roberts(rxRA_txR + rxRB_txR + rxRC_txR + rxRD_txR, wav_rxR_txR, sim.lightspeed, sim.f_emitted)

        deld_rxLR_txL[j] = d_rxL_txL[j] - d_rxR_txL[j]
        deld_rxLR_txR[j] = d_rxL_txR[j] - d_rxR_txR[j]

        thd = 1e-5; # just to avoid messing up the graphs when aoa detection is too bad.
        # assume that the VLC subsystem correctly decodes the signal (mark that the delayed signal 
        # is decoded, so the delay isn't actually measured, it's implicitly present on the RX signal)
        wav_rxL_txL = np.sin(2*np.pi*sim.f_emitted*(simulation_time - delayLL_sigTime))
        wav_rxR_txL = np.sin(2*np.pi*sim.f_emitted*(simulation_time - delayLR_sigTime))
        wav_rxL_txR = np.sin(2*np.pi*sim.f_emitted*(simulation_time - delayRL_sigTime))
        wav_rxR_txR = np.sin(2*np.pi*sim.f_emitted*(simulation_time - delayRR_sigTime))

        aoa_rxL_txL[j] = measure_bearing(rxLA_txL, rxLB_txL, rxLC_txL, rxLD_txL, wav_rxL_txL, sim.rx_config_bundle["f_QRX"], thd)
        aoa_rxR_txL[j] = measure_bearing(rxRA_txL, rxRB_txL, rxRC_txL, rxRD_txL, wav_rxR_txL, sim.rx_config_bundle["f_QRX"], thd)
        aoa_rxL_txR[j] = measure_bearing(rxLA_txR, rxLB_txR, rxLC_txR, rxLD_txR, wav_rxL_txR, sim.rx_config_bundle["f_QRX"], thd)
        aoa_rxR_txR[j] = measure_bearing(rxRA_txR, rxRB_txR, rxRC_txR, rxRD_txR, wav_rxR_txR, sim.rx_config_bundle["f_QRX"], thd)

        delaoa_rxLR_txL[j] = aoa_rxL_txL[j] - aoa_rxR_txL[j];
        delaoa_rxLR_txR[j] = aoa_rxL_txR[j] - aoa_rxR_txR[j];

        est_x_directbearing[j], est_y_directbearing[j] = classicalfix_directbearing_mle(aoa_rxL_txL[j], aoa_rxR_txL[j], L_ego)
        est_x_diffbearing[j], est_y_diffbearing[j]     = classicalfix_diffbearing_mle(delaoa_rxLR_txL[j], delaoa_rxLR_txR[j], L_ego)
        est_x_directrange[j], est_y_directrange[j]     = classicalfix_directrange_mle(d_rxL_txL[j], d_rxR_txL[j], L_ego)
        est_x_diffrange[j], est_y_diffrange[j]         = classicalfix_diffrange_mle(deld_rxLR_txL[j], deld_rxLR_txR[j], L_ego)

    e_dLL = np.sqrt(x[0]**2 +y[0]**2) - d_rxL_txL
    e_dRL = np.sqrt((x[0]-L_ego)**2 +y[0]**2) - d_rxR_txL
    e_dLR = np.sqrt((x[0]+L_target)**2 +y[0]**2) - d_rxL_txR
    e_dRR = np.sqrt((x[0]+L_target-L_ego)**2 +y[0]**2) - d_rxR_txR
    
    e_deld_L = (np.sqrt(x[0]**2 +y[0]**2) - np.sqrt((x[0]-L_ego)**2 +y[0]**2)) - deld_rxLR_txL
    e_deld_R = (np.sqrt((x[0]+L_target)**2 +y[0]**2) - np.sqrt((x[0]+L_target-L_ego)**2 +y[0]**2)) - deld_rxLR_txR
    
    e_aLL = np.rad2deg(np.arctan2(x[0],y[0])) - aoa_rxL_txL
    e_aRL = np.rad2deg(np.arctan2(x[0]-L_ego,y[0])) - aoa_rxR_txL
    e_aLR = np.rad2deg(np.arctan2(x[0]+L_target,y[0])) - aoa_rxL_txR
    e_aRR = np.rad2deg(np.arctan2(x[0]+L_target-L_ego,y[0])) - aoa_rxR_txR
    
    e_dela_L = (np.rad2deg(np.arctan2(x[0],y[0])) - np.rad2deg(np.arctan2(x[0]-L_ego,y[0]))) - delaoa_rxLR_txL
    e_dela_R = (np.rad2deg(np.arctan2(x[0]+L_target,y[0])) - np.rad2deg(np.arctan2(x[0]+L_target-L_ego,y[0]))) - delaoa_rxLR_txR
    
    crlb_x_directbearing, crlb_y_directbearing = classicalfix_directbearing_crlb( np.std(np.deg2rad(e_aLL)), np.std(np.deg2rad(e_aRL)), x[0], y[0], L_ego) 
    crlb_x_diffbearing, crlb_y_diffbearing     = classicalfix_diffbearing_crlb( np.std(np.deg2rad(e_dela_L)), np.std(np.deg2rad(e_dela_R)), x[0], y[0], L_ego) 
    crlb_x_directrange, crlb_y_directrange     = classicalfix_directrange_crlb(np.std(e_dLL), np.std(e_dRL), x[0], y[0], L_ego) 
    crlb_x_diffrange, crlb_y_diffrange         = classicalfix_diffrange_crlb(np.std(e_deld_L), np.std(e_deld_R), x[0], y[0], L_ego) 
        
    return  est_x_directbearing, est_y_directbearing, est_x_directrange, est_y_directrange, \
            est_x_diffbearing, est_y_diffbearing, est_x_diffrange, est_y_diffrange, \
            crlb_x_directbearing, crlb_y_directbearing, crlb_x_directrange, crlb_y_directrange, \
            crlb_x_diffbearing, crlb_y_diffbearing, crlb_x_diffrange, crlb_y_diffrange

### Run simulation over the grid

In [4]:
est_x_directbearing = np.zeros((numpts, iterations))
est_y_directbearing = np.zeros((numpts, iterations))
est_x_directrange = np.zeros((numpts, iterations))
est_y_directrange = np.zeros((numpts, iterations))
est_x_diffbearing = np.zeros((numpts, iterations))
est_y_diffbearing = np.zeros((numpts, iterations))
est_x_diffrange = np.zeros((numpts, iterations))
est_y_diffrange = np.zeros((numpts, iterations))
crlb_x_directbearing = np.zeros((numpts))
crlb_y_directbearing = np.zeros((numpts))
crlb_x_directrange = np.zeros((numpts))
crlb_y_directrange = np.zeros((numpts))
crlb_x_diffbearing = np.zeros((numpts))
crlb_y_diffbearing = np.zeros((numpts))
crlb_x_diffrange = np.zeros((numpts))
crlb_y_diffrange = np.zeros((numpts))

for idx, _ in enumerate(tqdm(x)):
    xRL_to_rxL_rep = np.ones(( int(f_vehicle/vlp_rate) ))*x[idx]
    yRL_to_rxL_rep = np.ones(( int(f_vehicle/vlp_rate) ))*y[idx]
    hdg_rep        = np.ones(( int(f_vehicle/vlp_rate) ))*hdg[idx]

    est_x_directbearing[idx,:], \
    est_y_directbearing[idx,:], \
    est_x_directrange[idx,:], \
    est_y_directrange[idx,:], \
    est_x_diffbearing[idx,:], \
    est_y_diffbearing[idx,:], \
    est_x_diffrange[idx,:], \
    est_y_diffrange[idx,:], \
    crlb_x_directbearing[idx], \
    crlb_y_directbearing[idx], \
    crlb_x_directrange[idx], \
    crlb_y_directrange[idx], \
    crlb_x_diffbearing[idx], \
    crlb_y_diffbearing[idx], \
    crlb_x_diffrange[idx], \
    crlb_y_diffrange[idx] \
    = simrun(xRL_to_rxL_rep, yRL_to_rxL_rep, hdg_rep,
             s_adc_clock, sim, iterations);

100%|███████████████████████████████████████████| 20/20 [02:35<00:00,  7.77s/it]


### Save

In [5]:
np.savez("results/survey_es3.npz",
         iterations = iterations,
         x=x, y=y, hdg=hdg,
         est_x_directbearing=est_x_directbearing,
         est_y_directbearing=est_y_directbearing,
         est_x_directrange=est_x_directrange,
         est_y_directrange=est_y_directrange,
         est_x_diffbearing=est_x_diffbearing,
         est_y_diffbearing=est_y_diffbearing,
         est_x_diffrange=est_x_diffrange,
         est_y_diffrange=est_y_diffrange,
         crlb_x_directbearing=crlb_x_directbearing,
         crlb_y_directbearing=crlb_y_directbearing,
         crlb_x_directrange=crlb_x_directrange,
         crlb_y_directrange=crlb_y_directrange,
         crlb_x_diffbearing=crlb_x_diffbearing,
         crlb_y_diffbearing=crlb_y_diffbearing,
         crlb_x_diffrange=crlb_x_diffrange,
         crlb_y_diffrange=crlb_y_diffrange)